Replicate [Dynamic Return Dependencies Across Industries: A Machine Learning Approach](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3120110&download=yes) by David Rapach, Jack Strauss, Jun Tu and Guofu Zhou.

1) Use industry returns from [Ken French](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html)

2) Forecast (for example) this month's Chemical industry return using last month's returns from all 30 industries 

3) Use LASSO for predictor subset selection over the entire 1960-2016 period to determine that e.g. Beer is predicted by Food, Clothing, Coal

4) Use those predictors and simple linear regression to predict returns

5) Generate portfolios and run backtests.

- Predictor selection - finds same predictors except 2 industries. Possibly use of AICc instead of AIC (don't see an sklearn implementation that uses AICc)

- Prediction by industry - R-squareds line up pretty closely

- Portfolio performance, similar ballpark results. Since prediction is similar but return profile is different, must be some difference in portfolio construction. (am taking equal weight top 6 predicted as long and bottom 6 as short, every month)

- For some reason their mean returns don't line up to geometric mean annualized, they seem to be calculating something different.

- But it does replicate closely and perform pretty well

In [1]:
# run MLP with and without scaling, see if you get better prediction

import os
import sys
import warnings
import numpy as np
import pandas as pd
import time 
import copy
import random
from itertools import product

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy numpy warnings

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression, Lasso, lasso_path, lars_path, LassoLarsIC
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

import ffn
%matplotlib inline

import plotly as py
# print (py.__version__) # requires version >= 1.9.0
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

random.seed(1764)
np.random.seed(1764)


In [2]:
print("Loading data...")
data = pd.read_csv("30_Industry_Portfolios.csv")
data = data.set_index('yyyymm')
industries = list(data.columns)
# map industry names to col nums
ind_reverse_dict = dict([(industries[i], i) for i in range(len(industries))])

rfdata = pd.read_csv("F-F_Research_Data_Factors.csv")
rfdata = rfdata.set_index('yyyymm')
data['rf'] = rfdata['RF']

# subtract risk-free rate
# create a response variable led by 1 period to predict
for ind in industries:
    data[ind] = data[ind] - data['rf']

for ind in industries:
    data[ind+".3m"] = pd.rolling_mean(data[ind],3)
    
#for ind in industries:
#    data[ind+".6m"] = pd.rolling_mean(data[ind],6)

#for ind in industries:
#    data[ind+".12m"] = pd.rolling_mean(data[ind],12)
    
for ind in industries:
    data[ind+".lead"] = data[ind].shift(-1)

data = data.loc[data.index[data.index > 195911]]
data = data.drop(columns=['rf'])    
data = data.dropna(axis=0, how='any')

nresponses = len(industries)
npredictors = data.shape[1]-nresponses

predictors = list(data.columns[:npredictors])
predictor_reverse_dict = dict([(predictors[i], i) for i in range(len(predictors))])

responses = list(data.columns[-nresponses:])
response_reverse_dict = dict([(responses[i], i) for i in range(len(responses))])

print(data.shape)

data[['Food', 'Food.lead']]


Loading data...
(697, 90)


,Food,Food.lead
yyyymm,,
195912,2.01,-4.49
196001,-4.49,3.35
196002,3.35,-1.67
196003,-1.67,1.17
196004,1.17,8.20
196005,8.20,5.39
196006,5.39,-2.11
196007,-2.11,4.57
196008,4.57,-3.88


In [3]:
# exclude 2017 and later to tie to paper
#data = data.loc[data.index[data.index < 201701]]
data = data.loc[data.index[data.index > 195911]]
data


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
195912,2.01,0.35,-3.02,1.64,7.29,0.67,1.87,-1.97,3.08,0.74,...,0.62,-6.18,-7.93,-9.41,-4.31,-5.33,-6.09,-10.08,-4.68,-3.98
196001,-4.49,-5.71,-2.05,1.21,-5.47,-7.84,-8.53,-6.68,-10.03,-4.77,...,8.07,9.13,5.09,3.00,-0.94,1.42,4.00,1.81,-0.98,6.32
196002,3.35,-2.14,2.27,4.23,2.39,9.31,1.44,-0.02,-0.74,0.32,...,-0.21,-0.31,3.34,-2.43,-4.99,-1.37,-0.13,-3.88,0.05,-2.43
196003,-1.67,-2.94,-0.18,-0.65,2.18,-0.56,-2.59,1.26,-2.75,-6.79,...,-1.24,7.14,1.77,0.41,-2.13,0.45,-0.53,8.86,-0.64,0.55
196004,1.17,-2.16,1.35,6.46,-1.17,-1.27,0.21,1.49,-5.53,-1.10,...,3.05,-1.75,11.90,2.85,0.90,1.65,3.11,0.80,-0.45,1.02
196005,8.20,-0.52,2.44,7.28,11.67,7.74,1.74,13.50,3.40,2.10,...,-0.58,-8.07,2.39,3.50,2.17,5.96,3.41,1.03,3.72,6.41
196006,5.39,0.47,4.73,2.24,0.02,6.38,-1.59,-0.40,0.45,4.04,...,-0.03,2.84,-2.02,-4.10,-3.11,-6.16,-2.99,-1.25,0.09,-5.95
196007,-2.11,-0.79,4.60,-4.72,0.23,-0.60,-1.10,-3.99,-6.80,-3.14,...,6.94,5.69,2.71,1.18,1.98,4.51,2.85,2.05,3.47,3.48
196008,4.57,3.24,5.20,7.16,3.63,5.09,3.34,2.29,1.17,-0.84,...,-6.07,-3.53,-7.61,-7.37,-7.07,-8.44,-8.57,-1.90,-5.78,-4.21


In [4]:
desc = data.describe()
desc
# min, max line up with Table 1

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
count,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,...,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000
mean,0.688666,0.727030,0.985079,0.732095,0.532253,0.564333,0.690387,0.665825,0.552367,0.687145,...,0.515968,0.729928,0.622970,0.534806,0.601090,0.631076,0.698235,0.728766,0.637547,0.396628
std,4.308660,5.058992,6.032324,7.128170,5.780362,4.728000,6.355251,4.897557,5.482363,6.970961,...,4.607931,6.486956,6.698787,5.021876,5.707154,5.571040,5.334178,6.065564,5.381389,5.771655
min,-18.150000,-20.190000,-25.320000,-33.400000,-26.560000,-22.240000,-31.500000,-21.060000,-28.600000,-33.110000,...,-16.440000,-28.670000,-32.070000,-27.740000,-28.500000,-29.250000,-29.740000,-31.890000,-22.530000,-28.090000
25%,-1.630000,-2.080000,-2.740000,-3.390000,-2.600000,-2.030000,-2.800000,-2.230000,-2.750000,-3.170000,...,-2.110000,-3.050000,-3.220000,-2.400000,-2.780000,-2.560000,-2.380000,-2.840000,-2.400000,-2.930000
50%,0.740000,0.750000,1.270000,0.940000,0.510000,0.750000,0.700000,0.760000,0.720000,0.640000,...,0.590000,1.010000,0.670000,0.710000,0.900000,0.940000,0.540000,1.080000,0.870000,0.540000
75%,3.070000,3.690000,4.660000,5.260000,3.640000,3.540000,4.310000,3.550000,3.760000,4.480000,...,3.360000,4.260000,4.630000,3.460000,4.040000,3.880000,3.980000,4.300000,4.000000,4.200000
max,19.890000,25.510000,32.380000,34.520000,33.130000,18.220000,31.790000,29.010000,21.680000,59.030000,...,21.220000,23.380000,24.660000,21.000000,18.500000,17.530000,26.490000,27.380000,20.590000,19.960000


In [5]:
# annualized returns don't match Table 1, oddly
# geometric mean, annualized
pd.DataFrame((np.prod(data/100 + 1)**(12.0/len(data))-1)[:30], columns=['Mean Ann. Return'])

,Mean Ann. Return
Food,0.073929
Beer,0.074309
Smoke,0.100741
Games,0.058342
Books,0.044662
Hshld,0.055568
Clths,0.060067
Hlth,0.067552
Chems,0.049242
Txtls,0.054817


In [6]:
# try this way, arithmetic mean then annualize (not very correct)
#print(pd.DataFrame(((desc.loc['mean']/100+1)**12-1)[:30]))
#nope

# same
pd.DataFrame(((1 + np.mean(data, axis=0)/100)**12 -1)[:30], columns=['Mean Ann. Return'])

,Mean Ann. Return
Food,0.085843
Beer,0.090818
Smoke,0.124829
Games,0.091476
Books,0.065774
Hshld,0.069862
Clths,0.086066
Hlth,0.082891
Chems,0.068335
Txtls,0.085646


In [7]:
#annualized volatility 
pd.DataFrame((desc.loc['std']*np.sqrt(12))[:30].round(2))
# lines up with table 1

,std
Food,14.93
Beer,17.52
Smoke,20.90
Games,24.69
Books,20.02
Hshld,16.38
Clths,22.02
Hlth,16.97
Chems,18.99
Txtls,24.15


In [8]:
# Run LASSO, then OLS on selected variables

# skip last row to better match published r-squared
# looks like they forecast actuals 1960-2016 using 1959m12 to 2016m11
# not exact matches to Table 2 R-squared but almost within rounding error 
X = data.values[:-1,:npredictors]
Y = data.values[:-1,-nresponses:]
nrows = X.shape[0]
X.shape

(696, 60)

In [9]:
def subset_selection(X, Y, model_aic, verbose=False):
    
    global responses
    global response_reverse_dict
    global predictors
    global predictor_reverse_dict
    
    coef_dict = {}
    for response_index, response in enumerate(responses):
        y = Y[:,response_reverse_dict[response]]
        
        model_aic.fit(X, y)

        coef_dict[response] = [predstr for i, predstr in enumerate(predictors) if model_aic.coef_[i] !=0]
        #y_response = model_aic.responseict(X)
        # print ("In-sample LASSO R-squared: %.6f" % r2_score(y, y_response))
        if verbose:
            print("LASSO variables selected for %s: " % response)
            print(coef_dict[response])
        
        if not coef_dict[response]:
            if verbose:
                print("No coefs selected for " + response + ", using all")
                print("---")
            coef_dict[response] = predictors            
        # fit OLS vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        if verbose:
            print("Running OLS for " + response + " against " + str(coef_dict[response]))
            # col nums of selected responses
            predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
            model_ols = LinearRegression()
            model_ols.fit(X[:, predcols], y)
            y_pred = model_ols.predict(X[:, predcols])
            print ("In-sample OLS R-squared: %.2f" % (100 * r2_score(y, y_pred)))
            print("---")
            
    return coef_dict

coef_dict = subset_selection(X, Y, LassoLarsIC(criterion='aic'), verbose=True)

# These subsets line up closely with Table 2
# except Clths, Whlsl, we get different responses

LASSO variables selected for Food.lead: 
['Clths', 'Coal', 'Oil', 'Util', 'Rtail', 'Meals', 'Beer.3m', 'Mines.3m', 'Util.3m']
Running OLS for Food.lead against ['Clths', 'Coal', 'Oil', 'Util', 'Rtail', 'Meals', 'Beer.3m', 'Mines.3m', 'Util.3m']
In-sample OLS R-squared: 4.38
---
LASSO variables selected for Beer.lead: 
['Food', 'Clths', 'Coal', 'Beer.3m', 'Hlth.3m', 'Util.3m']
Running OLS for Beer.lead against ['Food', 'Clths', 'Coal', 'Beer.3m', 'Hlth.3m', 'Util.3m']
In-sample OLS R-squared: 4.16
---
LASSO variables selected for Smoke.lead: 
['Txtls', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Trans', 'Food.3m', 'Beer.3m', 'Hshld.3m', 'Clths.3m', 'Chems.3m', 'ElcEq.3m', 'Mines.3m', 'Oil.3m', 'Util.3m', 'Servs.3m', 'Paper.3m', 'Other.3m']
Running OLS for Smoke.lead against ['Txtls', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Trans', 'Food.3m', 'Beer.3m', 'Hshld.3m', 'Clths.3m', 'Chems.3m', 'ElcEq.3m', 'Mines.3m', 'Oil.3m', 'Util.3m', 'Servs.3m', 'Paper.3m', 'Other.3m']
I

In [10]:
# same predictors selected for all but 2 response vars
# use predictors from paper to match results
if False: # turn off/on
    coef_dict = {}
    coef_dict['Food.lead'] = ['Clths', 'Coal', 'Util', 'Rtail']
    coef_dict['Beer.lead'] = ['Food', 'Clths', 'Coal']
    coef_dict['Smoke.lead'] = ['Txtls', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
    coef_dict['Games.lead'] = ['Books', 'Clths', 'Coal', 'Fin']
    coef_dict['Books.lead'] = ['Games', 'Books', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Fin']
    coef_dict['Hshld.lead'] = ['Clths', 'Coal', 'Rtail']
    coef_dict['Clths.lead'] = ['Books', 'Clths', 'Chems', 'Steel', 'ElcEq', 'Carry',  'Coal', 'Oil', 'Util','Telcm', 'Servs', 'BusEq', 'Rtail']
    # Running OLS for Clths against ['Clths', 'Coal', 'Oil', 'Servs', 'Rtail']
    coef_dict['Hlth.lead'] = ['Books', 'Mines', 'Coal', 'Util']
    coef_dict['Chems.lead'] = ['Clths']
    coef_dict['Txtls.lead'] = ['Clths', 'Autos', 'Coal', 'Oil', 'Rtail', 'Fin']
    coef_dict['Cnstr.lead'] = ['Clths', 'Coal', 'Oil', 'Util', 'Trans', 'Rtail', 'Fin']
    coef_dict['Steel.lead'] = ['Fin']
    coef_dict['FabPr.lead'] = ['Trans', 'Fin']
    coef_dict['ElcEq.lead'] = ['Fin']
    coef_dict['Autos.lead'] = ['Hshld', 'Clths', 'Coal', 'Oil', 'Util', 'BusEq', 'Rtail', 'Fin']
    coef_dict['Carry.lead'] = ['Trans']
    coef_dict['Mines.lead'] = []
    coef_dict['Coal.lead'] = ['Beer', 'Smoke', 'Books', 'Autos', 'Coal', 'Oil', 'Paper', 'Rtail']
    coef_dict['Oil.lead'] = ['Beer', 'Hlth', 'Carry']
    coef_dict['Util.lead'] = ['Food', 'Beer', 'Smoke', 'Hshld', 'Hlth', 'Cnstr', 'FabPr', 'Carry', 'Mines', 'Oil', 'Util', 'Telcm', 'BusEq', 'Whlsl', 'Fin', 'Other']
    coef_dict['Telcm.lead'] = ['Beer', 'Smoke', 'Books', 'Hshld', 'Cnstr', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Meals', 'Fin']
    coef_dict['Servs.lead'] = ['Smoke', 'Books', 'Steel', 'Oil', 'Util', 'Fin']
    coef_dict['BusEq.lead'] = ['Smoke', 'Books', 'Util']
    coef_dict['Paper.lead'] = ['Clths', 'Coal', 'Oil', 'Rtail', 'Fin']
    coef_dict['Trans.lead'] = ['Fin']
    coef_dict['Whlsl.lead'] = ['Food', 'Beer', 'Smoke', 'Books', 'Hlth', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Fin', 'Other']
    # Running OLS for Whlsl against ['Food', 'Smoke', 'Books', 'Carry', 'Coal', 'Oil', 'Util', 'Servs', 'Fin', 'Other']
    coef_dict['Rtail.lead'] = ['Rtail']
    coef_dict['Meals.lead'] = ['Smoke', 'Books', 'Clths', 'Steel', 'Carry', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Meals', 'Fin']
    coef_dict['Fin.lead'] = ['Fin']
    coef_dict['Other.lead'] = ['Clths', 'Fin']


In [11]:
def predict_with_subsets(X, Y, model, coef_dict, verbose=False):

    global responses
    global response_reverse_dict
    
    scores = []
    for response in responses:
        y = Y[:,response_reverse_dict[response]]

#        print("LASSO variables selected for %s: " % pred)
#        print(coef_dict[pred])
        
        if not coef_dict[response]:
            if verbose:
                print("No coefs selected for " + response)
 #           print("---")
            continue
        # fit model vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        #print("Running model for " + pred + " against " + str(coef_dict[pred]))
        # col nums of selected predictors
        predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
        model.fit(X[:, predcols], y)
        y_pred = model.predict(X[:, predcols])
        score = r2_score(y, y_pred)
        scores.append(score)
        if verbose:
            print ("In-sample R-squared: %.4f for %s against %s" % (score, response, str(coef_dict[response])))
#        print("---")
    
    if verbose:
        print("Mean R-squared: %.4f" % np.mean(np.array(scores)))
    return np.mean(np.array(scores))
    

predict_with_subsets(X, Y, LinearRegression(), coef_dict, verbose=True)


In-sample R-squared: 0.0438 for Food.lead against ['Clths', 'Coal', 'Oil', 'Util', 'Rtail', 'Meals', 'Beer.3m', 'Mines.3m', 'Util.3m']
In-sample R-squared: 0.0416 for Beer.lead against ['Food', 'Clths', 'Coal', 'Beer.3m', 'Hlth.3m', 'Util.3m']
In-sample R-squared: 0.1123 for Smoke.lead against ['Txtls', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Trans', 'Food.3m', 'Beer.3m', 'Hshld.3m', 'Clths.3m', 'Chems.3m', 'ElcEq.3m', 'Mines.3m', 'Oil.3m', 'Util.3m', 'Servs.3m', 'Paper.3m', 'Other.3m']
In-sample R-squared: 0.0553 for Games.lead against ['Books', 'Clths', 'Coal', 'Fin', 'Steel.3m']
In-sample R-squared: 0.1055 for Books.lead against ['Games', 'Books', 'Clths', 'Autos', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Meals', 'Fin', 'Books.3m', 'Chems.3m', 'Steel.3m', 'ElcEq.3m', 'Autos.3m', 'Mines.3m', 'Coal.3m', 'Util.3m', 'Telcm.3m', 'Servs.3m', 'Paper.3m']
In-sample R-squared: 0.0383 for Hshld.lead against ['Clths', 'Coal', 'Rtail', 'Steel.3m', 'Util.3m']
In-sample R-squ

0.066816985873582

In [12]:
def fit_predict(X, Y, model, coef_dict=None):
    """for backtest, fit Ys v. X using n-1 rows
    predict Ys on X using nth row
    return a prediction for month n+1 using X for final month"""
    
    global responses
    global response_reverse_dict
    
    # keep last row to predict against
    X_predict = X[-1]
    X_predict = X_predict.reshape(1,X.shape[1])
    # fit on remaining rows
    X_fit = X[:-1]
    Y_fit = Y[:-1]

    # if no coef_dict select predictors into coef_dict
    if coef_dict is None:
        coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'))

    predictions = []
    for response in responses:
        if not coef_dict[response]:
            predictions.append(0.0)
            continue
        # column predexes to fit against each other
        predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
        responsecol = response_reverse_dict[response]
        model.fit(X_fit[:, predcols], Y_fit[:,responsecol])
        y_pred = model.predict(X_predict[:,predcols])        
        predictions.append(y_pred[0])
        
    return predictions

#    return np.argsort(predictions)

X = data.values[:,:npredictors]
Y = data.values[:, -nresponses:]
model = LinearRegression()
predictions = fit_predict(X, Y, model, coef_dict)
predictions

[-0.8536473674516295,
 -0.7708285578179694,
 -2.4280971602836754,
 -0.07882299101500101,
 -2.023396225788131,
 0.06315081701329256,
 -0.8533691403459845,
 -1.079652578340466,
 -0.25344454364601343,
 -0.5460795367539997,
 -1.3130952344726021,
 0.35380818707447625,
 -2.6600182569991935,
 -1.2373953026559863,
 -1.3830369632222037,
 -1.807316312559728,
 -4.170719450208045,
 0.6587990973550054,
 0.8733823800768382,
 -1.4330584604736534,
 -0.9183593929203996,
 -0.6698044320601328,
 -0.7549309496940442,
 -0.2139496959653665,
 -0.8270259584809956,
 -1.4676526087862514,
 -1.8447633904066465,
 -2.1956037974064464,
 0.6624203768057081,
 -1.5039208535103081]

In [13]:
# 197001 = 121
STARTMONTH = 121
print(X[STARTMONTH])
print(data.iloc[STARTMONTH][:30])

[ -3.34        -1.95        -7.59        -7.76       -12.05
  -7.5         -5.69        -7.71        -7.37        -5.26
  -9.84        -6.31        -7.15        -6.89        -9.35
 -12.49        -2.34        -0.77       -12.16        -4.83
  -3.16       -11.17        -9.73        -8.89        -8.17
  -8.28        -6.31       -13.12        -9.78        -6.2
  -2.28666667  -2.18        -3.36        -7.00333333  -6.82
  -3.37666667  -5.32666667  -1.41        -5.58666667  -5.43333333
  -6.02666667  -4.45        -4.68333333  -4.67666667  -5.93666667
  -9.37        -2.48         2.38666667  -6.98666667  -3.96666667
  -2.88333333  -4.91666667  -4.43        -4.87666667  -8.43
  -6.69666667  -4.78333333  -7.30333333  -6.68333333  -6.49      ]
Food     -3.34
Beer     -1.95
Smoke    -7.59
Games    -7.76
Books   -12.05
Hshld    -7.50
Clths    -5.69
Hlth     -7.71
Chems    -7.37
Txtls    -5.26
Cnstr    -9.84
Steel    -6.31
FabPr    -7.15
ElcEq    -6.89
Autos    -9.35
Carry   -12.49
Mines    -2.34
C

In [1]:
# predict all months starting STARTMONTH
# initialize predictions matrix P

def run_backtest(X, Y, model, coef_dict=None, startmonth=0, minmaxscale=False):
    global P
    global R 

    P = np.zeros_like(Y)
    count = 0
    for month_index in range(startmonth, X.shape[0]+1):
        # 0 to month_index-1
        Xscale = X.copy()
        Yscale = Y.copy()

        if minmaxscale:
            # minmaxscale each row
            for i in range(Xscale.shape[0]):
                Xscale[i] = Xscale[i] - np.min(Xscale[i])
                Xscale[i] = Xscale[i]/np.max(Xscale[i])
                
            for i in range(Yscale.shape[0]):
                Yscale[i] = Yscale[i] - np.min(Yscale[i])
                Yscale[i] = Yscale[i]/np.max(Yscale[i])
        
        predictions = fit_predict(Xscale[:month_index, :], 
                                  Yscale[:month_index], 
                                  model,
                                  coef_dict)
        try:
            P[month_index]= predictions
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
            sys.stdout.flush()
        except IndexError:
            # I want to run the fit and see the R-squared on full dataset
            # but we are storing the predictions in row of the month predicted
            # so we have no row to store the last prediction (2017-01)
            print("\nlast prediction not stored")

    mse = np.mean((P[startmonth:]-Xscale[startmonth:,-nresponses:])**2)
    print("MSE across all predictions: %.4f" % mse)
    print("Variance: %.4f" % (np.mean(Yscale[startmonth:,-nresponses]**2)))
    print("R-squared: %.4f" % (1- mse/np.mean(Yscale[startmonth:,-nresponses]**2)))

    R = np.zeros(P.shape[0])
    numstocks = 6 # top quintile (and bottom)

    for month_index in range(startmonth, X.shape[0]):
        # get indexes of sorted smallest to largest
        select_array = np.argsort(P[month_index])
        # leftmost 6
        short_indexes = select_array[:numstocks]
        # rightmost 6
        long_indexes = select_array[-numstocks:]
        # compute equal weighted long/short return
        R[month_index] = np.mean(X[month_index, long_indexes])/2 - np.mean(X[month_index, short_indexes])/2

    results = R[startmonth:]

    index = pd.date_range('01/01/1970',periods=results.shape[0], freq='M')
    perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
    perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)

    stats = perfdata['Equity'].calc_stats()

    retframe = pd.DataFrame([stats.stats.loc['start'],
                             stats.stats.loc['end'],
                             stats.stats.loc['cagr'],
                             stats.stats.loc['yearly_vol'],
                             stats.stats.loc['yearly_sharpe'],
                             stats.stats.loc['max_drawdown'],
                             ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=564, annualize=False),
                            ],
                            index = ['start',
                                     'end',
                                     'cagr',
                                     'yearly_vol',
                                     'yearly_sharpe',
                                     'max_drawdown',
                                     'sortino',
                                    ],
                            columns=['Value'])   
    return retframe


In [34]:
print("%s Starting" % (time.strftime("%H:%M:%S")))
model = LinearRegression()
run_backtest(X, Y, model, coef_dict, startmonth=STARTMONTH, minmaxscale=False)

20:38:56 Starting
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................
last prediction not stored
MSE across all predictions: 0.0530


,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0505986
yearly_vol,0.0586134
yearly_sharpe,0.871087
max_drawdown,-0.101667
sortino,0.48858


In [43]:
# without minmaxscale
print("%s Starting" % (time.strftime("%H:%M:%S")))
model = LinearRegression()
run_backtest(X, Y, model, coef_dict, startmonth=STARTMONTH, minmaxscale=False)

23:32:45 Starting
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................
last prediction not stored
MSE across all predictions: 14.2270


,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.072308
yearly_vol,0.0901095
yearly_sharpe,0.840289
max_drawdown,-0.101805
sortino,0.719113


In [35]:
# double check results
#model = LinearRegression()
#R = run_backtest(X, Y, model, coef_dict_paper, startmonth=STARTMONTH, summary=False)
results = R[STARTMONTH:]
print(len(results))
#print(results)
print(np.mean(results))
print(np.std(results) * np.sqrt(12))
print(np.prod(1 + results / 100))
print(np.prod(1 + results / 100) ** (12.0/results.shape[0]))-1

576
0.422214988425926
5.433597396370643
10.560532923520368
0.05033246212836251


In [36]:
# run performance chart
perf = 100 * np.cumprod(1 + results / 100)

def mychart(perf):
    x_coords = np.linspace(1970, 2016, perf.shape[0])
    
    trace1 = Scatter(
        x = x_coords,
        y = perf,
        name = 'Growth of $1',    
    )

    layout = Layout(
        yaxis=dict(
            type='log',
            autorange=True
        )
    )
    plotdata = [trace1]
    
    fig = Figure(data=plotdata, layout=layout)
    
    iplot(fig)
    
mychart(perf)

In [37]:
# pass coef_dict as None
# fit_predict will do subset selection at each timestep using data it trains on
print("%s Starting" % (time.strftime("%H:%M:%S")))
model = LinearRegression()
run_backtest(X, Y, model, coef_dict=None, startmonth=STARTMONTH, minmaxscale=True)

20:39:23 Starting
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................
last prediction not stored
MSE across all predictions: 0.0543


,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0260419
yearly_vol,0.0494673
yearly_sharpe,0.535379
max_drawdown,-0.173108
sortino,0.231912


In [38]:
results = R[STARTMONTH:]
perf = 100 * np.cumprod(1 + results / 100)
mychart(perf)

In [20]:
def walkforward_xval (X, Y, model, coef_dict=None, minmaxscale=False):

    start = time.time()

    Xscale = X.copy()
    Yscale = Y.copy()

    if minmaxscale:
        # minmaxscale each row
        for i in range(Xscale.shape[0]):
            Xscale[i] = Xscale[i] - np.min(Xscale[i])
            Xscale[i] = Xscale[i]/np.max(Xscale[i])
            
        for i in range(Yscale.shape[0]):
            Yscale[i] = Yscale[i] - np.min(Yscale[i])
            Yscale[i] = Yscale[i]/np.max(Yscale[i])
            
    # generate k-folds
    n_splits = 5
    kf = KFold(n_splits=n_splits)
    kf.get_n_splits(Xscale)
    last_indexes = []
    for train_index, test_index in kf.split(Xscale):
        # use test_index as last index to train
        last_index = test_index[-1] + 1
        last_indexes.append(last_index)
    print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))

    print("%s Starting training" % (time.strftime("%H:%M:%S")))
    
    avg_bests = []
    for i in range(1, n_splits-1):

        models = []
        losses = []
        scores = []
        count = 0        
        # skip kfold 0 so you start with train 2x size of eval set
        last_train_index = last_indexes[i]
        last_xval_index = last_indexes[i+1]

        # set up train, xval
        # train from beginning to last_train_index        
        print("Training indexes 0 to %d" % (last_train_index-1))
        X_fit = Xscale[:last_train_index]
        Y_fit = Yscale[:last_train_index]
        # xval from last_train_index to last_xval_index
        print("Cross-validating indexes %d to %d" % (last_train_index, last_xval_index -1 ))
        X_xval = Xscale[last_train_index:last_xval_index]
        Y_xval = Yscale[last_train_index:last_xval_index]

        if coef_dict is None:
            print("Performing LASSO subset selection on training set")
            coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'), verbose=False)
        
        mse_list = []
        
        for response in responses:
            predcols = [predictor_reverse_dict[indstr] for indstr in coef_dict[response]]
            if len(predcols) == 0:
                continue
            responsecol = response_reverse_dict[response]
            
            fit = model.fit(X_fit[:,predcols], Y_fit[:,responsecol])
            # evaluate ... run prediction, calc MSE by industry, and average
            y_xval_pred = fit.predict(X_xval[:,predcols])
            mse_list.append(mean_squared_error(Y_xval[:,i], y_xval_pred))
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training" % (time.strftime("%H:%M:%S")))
            sys.stdout.flush()             
        # mean mse over industry ys for this fold
        xval_score = np.mean(np.array(mse_list))            

        # choose model with lowest xval loss
        print ("\n%s Xval MSE %f" % (time.strftime("%H:%M:%S"), xval_score))
        avg_bests.append(xval_score)
    
    print ("Last Xval loss %f" % (xval_score))
    # mean over folds
    avg_loss = np.mean(np.array(avg_bests))
    print ("Avg Xval loss %f" % avg_loss)
    print("--------------------------------------------------------------------------------")
    return (avg_loss, model)


In [21]:
# walk forward with LinearRegression to get a baseline MSE
model = LinearRegression()
walkforward_xval (X, Y, model, coef_dict=coef_dict, minmaxscale=True)


19:04:46 Generate splits [140, 280, 419, 558, 697]
19:04:46 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
19:04:46 Xval MSE 0.083287
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
19:04:46 Xval MSE 0.115757
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
19:04:46 Xval MSE 0.081577
Last Xval loss 0.081577
Avg Xval loss 0.093541
--------------------------------------------------------------------------------


(0.09354052111484512,
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))

In [22]:
model = MLPRegressor(hidden_layer_sizes=(2,2,2),
                     alpha=1.0,
                     activation='tanh',
                     max_iter=10000, 
                     tol=1e-10,
                     solver='lbfgs')
walkforward_xval (X, Y, model, coef_dict=coef_dict, minmaxscale=True)


19:04:46 Generate splits [140, 280, 419, 558, 697]
19:04:46 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
19:04:47 Xval MSE 0.077867
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
19:04:48 Xval MSE 0.110198
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
19:04:48 Xval MSE 0.079193
Last Xval loss 0.079193
Avg Xval loss 0.089086
--------------------------------------------------------------------------------


(0.08908620755601267,
 MLPRegressor(activation='tanh', alpha=1.0, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=(2, 2, 2), learning_rate='constant',
        learning_rate_init=0.001, max_iter=10000, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='lbfgs', tol=1e-10, validation_fraction=0.1,
        verbose=False, warm_start=False))

In [23]:
MODELPREFIX = "MLP"

n_hiddens = [1, 2, 3]
layer_sizes = [1, 2, 4, 8]
reg_penalties = [0.0, 0.001, 0.01, 0.1, 1]
hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))

experiments = {}

for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty)
    print("%s n_hidden_layers = %d, hidden_layer_size = %d, reg_penalty = %.6f" % 
          (time.strftime("%H:%M:%S"), n_hidden_layers, layer_size, reg_penalty))
    hls = tuple([layer_size]*n_hidden_layers)
    model = MLPRegressor(hidden_layer_sizes=hls,
                         alpha=reg_penalty,
                         activation='tanh',
                         max_iter=10000, 
                         tol=1e-10,
                         solver='lbfgs')
    
    score, model = walkforward_xval (X, Y, model, coef_dict=coef_dict, minmaxscale=True)

    experiments[key] = score


19:04:48 Running 60 experiments
19:04:48 Running experiment 1 of 60
19:04:48 n_hidden_layers = 1, hidden_layer_size = 1, reg_penalty = 0.000000
19:04:49 Generate splits [140, 280, 419, 558, 697]
19:04:49 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
19:04:49 Xval MSE 0.172606
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
19:04:50 Xval MSE 0.818340
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
19:04:51 Xval MSE 0.083405
Last Xval loss 0.083405
Avg Xval loss 0.358117
--------------------------------------------------------------------------------
19:04:51 Running experiment 2 of 60
19:04:51 n_hidden_layers = 1, hidden_layer_size = 1, reg_penalty = 0.001000
19:04:51 Generate splits [140, 280, 419, 558, 697]
19:04:51 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
19:04:5

..............................
19:07:05 Xval MSE 0.142937
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
19:07:14 Xval MSE 0.164141
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
19:07:24 Xval MSE 0.104014
Last Xval loss 0.104014
Avg Xval loss 0.137031
--------------------------------------------------------------------------------
19:07:24 Running experiment 14 of 60
19:07:24 n_hidden_layers = 1, hidden_layer_size = 4, reg_penalty = 0.100000
19:07:24 Generate splits [140, 280, 419, 558, 697]
19:07:24 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
19:07:28 Xval MSE 0.099754
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
19:07:32 Xval MSE 0.130136
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
19:07:38 Xval MSE 0.089504
Last Xval loss 0.089504

..............................
19:14:00 Xval MSE 0.111734
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
19:14:01 Xval MSE 0.079560
Last Xval loss 0.079560
Avg Xval loss 0.089742
--------------------------------------------------------------------------------
19:14:01 Running experiment 26 of 60
19:14:01 n_hidden_layers = 2, hidden_layer_size = 2, reg_penalty = 0.000000
19:14:01 Generate splits [140, 280, 419, 558, 697]
19:14:01 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
19:14:12 Xval MSE 0.397108
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
19:14:24 Xval MSE 0.234972
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
19:14:33 Xval MSE 0.088819
Last Xval loss 0.088819
Avg Xval loss 0.240300
--------------------------------------------------------------------------------
19:14:33 Runnin

..............................
19:29:44 Xval MSE 0.246529
Last Xval loss 0.246529
Avg Xval loss 0.311827
--------------------------------------------------------------------------------
19:29:44 Running experiment 38 of 60
19:29:44 n_hidden_layers = 2, hidden_layer_size = 8, reg_penalty = 0.010000
19:29:44 Generate splits [140, 280, 419, 558, 697]
19:29:44 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
19:30:34 Xval MSE 0.284638
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
19:31:36 Xval MSE 0.268429
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
19:32:50 Xval MSE 0.177225
Last Xval loss 0.177225
Avg Xval loss 0.243431
--------------------------------------------------------------------------------
19:32:50 Running experiment 39 of 60
19:32:50 n_hidden_layers = 2, hidden_layer_size = 8, reg_penalty = 0.100000
19:32:50 Generate spli

..............................
19:38:23 Xval MSE 0.077550
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
19:38:23 Xval MSE 0.110285
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
19:38:24 Xval MSE 0.079459
Last Xval loss 0.079459
Avg Xval loss 0.089098
--------------------------------------------------------------------------------
19:38:24 Running experiment 51 of 60
19:38:24 n_hidden_layers = 3, hidden_layer_size = 4, reg_penalty = 0.000000
19:38:24 Generate splits [140, 280, 419, 558, 697]
19:38:24 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
19:39:24 Xval MSE 0.276038
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
19:40:36 Xval MSE 0.267102
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
19:41:50 Xval MSE 0.196261
Last Xval loss 0.196261

In [24]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]
 
lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "loss"])
lossframe.sort_values(['loss'])

,n_hidden_layers,layer_size,reg_penalty,loss
35,3,1,1.000,0.088914
36,3,2,1.000,0.089098
9,3,4,1.000,0.089359
21,3,8,1.000,0.089665
1,2,1,1.000,0.089742
51,2,2,1.000,0.090344
0,3,1,0.100,0.090955
7,2,4,1.000,0.091094
44,1,1,1.000,0.091497
33,1,2,1.000,0.091737


In [25]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['loss'].mean())


,loss
n_hidden_layers,
1,342.437090
2,0.911002
3,0.174708


In [26]:
pd.DataFrame(lossframe.groupby(['layer_size'])['loss'].mean())


,loss
layer_size,
1,0.121655
2,2.296304
4,455.322653
8,0.289788


In [27]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['loss'].mean())


,loss
reg_penalty,
0.000,572.001699
0.001,0.190440
0.010,0.144275
0.100,0.110946
1.000,0.090641


In [28]:
def plot_matrix(lossframe, x_labels, y_labels, x_suffix="", y_suffix=""):

    pivot = lossframe.pivot_table(index=[x_labels], columns=[y_labels], values=['loss'])
    # specify labels as strings, to force it to use a discrete axis
    if lossframe[x_labels].dtype == np.float64 or lossframe[x_labels].dtype == np.float32:
        xaxis = ["%f %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    else:
        xaxis = ["%d %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    if lossframe[y_labels].dtype == np.float64 or lossframe[y_labels].dtype == np.float32:
        yaxis = ["%f %s" % (i, y_suffix) for i in pivot.index.values]
    else:
        yaxis = ["%d %s" % (i, y_suffix) for i in pivot.index.values]
        
    print(xaxis, yaxis)
    """plot a heat map of a matrix"""
    chart_width=640
    chart_height=480
    
    layout = Layout(
        title="%s v. %s" % (x_labels, y_labels),
        height=chart_height,
        width=chart_width,     
        margin=dict(
            l=150,
            r=30,
            b=120,
            t=100,
        ),
        xaxis=dict(
            title=x_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
        yaxis=dict(
            title=y_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
    )
    
    data = [Heatmap(z=pivot.values,
                    x=xaxis,
                    y=yaxis,
                    colorscale=[[0, 'rgb(0,0,255)', [1, 'rgb(255,0,0)']]],
                   )
           ]

    fig = Figure(data=data, layout=layout)
    return iplot(fig, link_text="")

plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" units", y_suffix=" layers")



(['1  units', '2  units', '4  units', '8  units'], ['1  layers', '2  layers', '3  layers'])


In [29]:
plot_matrix(lossframe, "n_hidden_layers", "reg_penalty", x_suffix="p", y_suffix=" layers")


(['0 p', '0 p', '0 p', '0 p', '1 p'], ['1.000000  layers', '2.000000  layers', '3.000000  layers'])


In [30]:
plot_matrix(lossframe, "reg_penalty", "layer_size", x_suffix=" units", y_suffix="p")


(['1.000000  units', '2.000000  units', '4.000000  units', '8.000000  units'], ['0 p', '0 p', '0 p', '0 p', '1 p'])


In [39]:
print("%s Starting" % (time.strftime("%H:%M:%S")))

model = MLPRegressor(hidden_layer_sizes=(1,1,1),
                     alpha=1.0,
                     activation='tanh',
                     max_iter=10000, 
                     tol=1e-10,
                     solver='lbfgs')
run_backtest(X, Y, model, startmonth=STARTMONTH, minmaxscale=True)
# lower MSE, worse performance
# linear is lucky? worse forecast accuracy gives better performance?

20:41:59 Starting
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................
last prediction not stored
MSE across all predictions: 0.0508


,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,-0.00537598
yearly_vol,0.0721702
yearly_sharpe,-0.0440373
max_drawdown,-0.385612
sortino,-0.024083


In [41]:
print("%s Starting" % (time.strftime("%H:%M:%S")))

model = MLPRegressor(hidden_layer_sizes=(1,1,1),
                     alpha=1.0,
                     activation='tanh',
                     max_iter=10000, 
                     tol=1e-10,
                     solver='lbfgs')
run_backtest(X, Y, model, startmonth=STARTMONTH, minmaxscale=False)
# runs slower without minmaxscale, optimization takes longer to converge

22:47:02 Starting
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................
last prediction not stored
MSE across all predictions: 14.1375


,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0162446
yearly_vol,0.0508186
yearly_sharpe,0.322345
max_drawdown,-0.368019
sortino,0.154886


In [40]:
print("%s Starting" % (time.strftime("%H:%M:%S")))
model = MLPRegressor(hidden_layer_sizes=(2,2,2),
                     alpha=1.0,
                     activation='tanh',
                     max_iter=10000, 
                     tol=1e-10,
                     solver='lbfgs')
run_backtest(X, Y, model, startmonth=STARTMONTH, minmaxscale=True)
# lower MSE, worse performance


20:46:27 Starting
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................
last prediction not stored
MSE across all predictions: 0.0508


,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.00135136
yearly_vol,0.0531628
yearly_sharpe,0.0439699
max_drawdown,-0.242301
sortino,0.0246606


In [42]:
print("%s Starting" % (time.strftime("%H:%M:%S")))
model = MLPRegressor(hidden_layer_sizes=(2,2,2),
                     alpha=1.0,
                     activation='tanh',
                     max_iter=10000, 
                     tol=1e-10,
                     solver='lbfgs')
run_backtest(X, Y, model, startmonth=STARTMONTH, minmaxscale=False)
# lower MSE, worse performance
# linear is lucky? worse forecast accuracy gives better performance?

23:16:53 Starting
...............................

KeyboardInterrupt: 